In [1]:
import os
import base64
import requests
import tempfile

from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
from supabase import create_client

load_dotenv()


def load_document(
    filename: str,
    document_type: str,
    chunking_type: str = "semantic",
    chunk_size: int = 1024,
    chunk_overlap: int = 200
):
    """
    Function use to load a PDF document onto a Supabase vector database.
    It will convert it into Markdown, split it by its headers, create an embedding for each chunk.
    Finally it will upload each embedded chunk to the 'embeddings' table.
    """
    document_types = [
        "certidao_registo_predial",
        "caderneta_predial",
        "licenca_utilizacao",
        "certidao_isencao",
        "certidao_infraestruturas",
        "ficha_tecnica_habitacao",
        "certificado_energetico",
        "planta_imovel",
        "documento_kyc",
        "documento_preferencia"
    ]

    if document_type not in document_types:
        raise Exception("Invalid document type")
    
    with tempfile.TemporaryDirectory() as tmp_dirname:
        # Parse the PDF and convert it to Markdown
        os.system(f"""marker_single "{filename}" "{tmp_dirname}" --batch_multiplier 1""")

        # Split the resulting Markdown into chunks
        resulting_folder_name = Path(filename).stem
        with open(f"{tmp_dirname}/{resulting_folder_name}/{resulting_folder_name}.md", "r") as f:
            doc = f.read()

            # Replace PDF images with its descriptions
            for image in Path(f"{tmp_dirname}/{resulting_folder_name}/").glob("*.png"):
                image_description = get_image_description(image)
                doc = doc.replace(f"![{image.name}]({image.name})", f"<Image Description>: {image_description}" if image_description else "")

            markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
                ("#", "Header 1"),
                ("##", "Header 2"),
                ("###", "Header 3"),
            ], strip_headers=False)
            chunks = markdown_splitter.split_text(doc)
            
            # Instantiate a Supabase and an OpenAI client
            supabase_client = create_client(os.environ.get("SUPABASE_URL"), os.environ.get("SUPABASE_KEY"))
            openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

            # Add an incrementing identifier to each chunk
            chunk_idx = 0
            final_chunks = []
            for chunk in chunks:
                # Divide header chunk into sub chunks if it falls below our predetermined chunk size
                if len(chunk.page_content) > chunk_size:
                    if chunking_type == "semantic":
                        text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY")), breakpoint_threshold_type="percentile") 
                        text_chunks = text_splitter.create_documents([chunk.page_content])
                    else:
                        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
                        text_chunks = text_splitter.split_documents([chunk])
                else:
                    text_chunks = [chunk]

                for text_chunk in text_chunks:
                    # Generate embeddings from OpenAI
                    response = openai_client.embeddings.create(
                        input=text_chunk.page_content,
                        model="text-embedding-3-small"
                    )

                    # Upload to table
                    supabase_client.table("embeddings").insert({
                        "content": text_chunk.page_content,
                        "embedding": response.data[0].embedding,
                        "metadata": {
                            "name": resulting_folder_name,
                            "chunk_idx": chunk_idx,
                            **chunk.metadata
                        },
                        "document_type": document_type
                    }).execute()

                    # Increment chunk index
                    chunk_idx = chunk_idx + 1
                    final_chunks.append(text_chunk)
    
    return final_chunks


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
  

def get_image_description(image_path: str):
    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ.get("OPENAI_API_KEY")}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Descreve em detalhe numa frase o que está nesta imagem usando português de Portugal. Se não consegues perceber devolve uma string vazia."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "low"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    try:
        response_dict = response.json()
        return response_dict["choices"][0]["message"]["content"]
    except Exception as e:
        print(e)
        return None

In [3]:
filename = "../data/certificado_energetico/4355_8432.pdf"
os.system(f"""marker_single "{filename}" "../dataTest" --batch_multiplier 1""")

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 2/2 [00:15<00:00,  7.83s/it]


Saved markdown to the ../dataTest/4355_8432 folder


0

In [2]:
chunks = load_document(
    filename="../data/certificado_energetico/4355_8432.pdf",
    document_type="certificado_energetico",
    chunking_type="recursive"
)
chunks

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 2/2 [00:15<00:00,  7.73s/it]


Saved markdown to the /var/folders/b4/t373qrvd4m76swgs_nb9vf9r0000gn/T/tmp2k71mijz/4355_8432 folder


[Document(page_content='Certificado Energético  \n<Image Description>: Certificação de edifícios relacionada com eficiência energética e qualidade do ar interior, apresentada com um logótipo colorido.  \nEdifício de Habitação SCE107727024 Válido até 15/07/2025 IDENTIFICAÇÃO POSTAL Morada ESTRADA DAS PEDRAS LAVRADAS, 28, Localidade SOBRAL DE SÃO MIGUEL\nFreguesia SOBRAL DE S. MIGUEL Concelho COVILHÃ GPS 40.212883, -7.741674 IDENTIFICAÇÃO PREDIAL/FISCAL Conservatória do Registo Predial de COVILHÃ Nº de Inscrição na Conservatória 1714 Artigo Matricial nº 637 Fração Autónoma INFORMAÇÃO ADICIONAL Área útil de Pavimento 100,44 m²\nEste certificado apresenta a classificação energética deste edifício ou fração. Esta classificação é calculada comparando o desempenho'),
 Document(page_content='Este certificado apresenta a classificação energética deste edifício ou fração. Esta classificação é calculada comparando o desempenho  \n<Image Description>: {"description": "Uma casa de dois andares com 

In [59]:
for idx, chunk in enumerate(chunks):
    if len(chunk.page_content) > 1024:
        print(idx, "Maior", len(chunk.page_content))
    else:
        print(idx, "Menor", len(chunk.page_content))

0 Maior 1148
1 Menor 365
2 Menor 191
3 Menor 761
4 Menor 994
5 Maior 2368
6 Menor 657
7 Maior 2509
8 Maior 1623
9 Maior 1041
10 Menor 192
11 Menor 791
12 Maior 14453
13 Maior 5796
14 Maior 2370
15 Maior 2646
16 Maior 3543


In [27]:
os.environ["DEFAULT_LANG"] = "Portuguese"
os.environ["OCR_ENGINE"] = "ocrmypdf"

filename = "../data/certificado_energetico/4355_8432.pdf"

# Parse the PDF and convert it to Markdown
os.system(f"""marker_single "{filename}" "dataTest" --batch_multiplier 1 --ocr_all_pages""")

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype
No languages specified for tesseract, defaulting to Portuguese.


Finding reading order: 100%|██████████| 2/2 [00:17<00:00,  8.55s/it]
Traceback (most recent call last):
  File "/Users/badbadnotvena/.pyenv/versions/chatbot-rag-venv/bin/marker_single", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/badbadnotvena/.pyenv/versions/3.12.2/envs/chatbot-rag-venv/lib/python3.12/site-packages/convert_single.py", line 34, in main
    full_text, images, out_meta = convert_single_pdf(fname, model_lst, max_pages=args.max_pages, langs=langs, batch_multiplier=args.batch_multiplier, start_page=args.start_page, ocr_all_pages=args.ocr_all_pages)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/badbadnotvena/.pyenv/versions/3.12.2/envs/chatbot-rag-venv/lib/python3.12/site-packages/marker/convert.py", line 126, in convert_single_pdf
    table_count = format_tables(pa

256

In [2]:
os.environ["DEFAULT_LANG"] = "Portuguese"
os.environ["OCR_ENGINE"] = "surya"

filename = "../data/certificado_energetico/4355_8432.pdf"

# Parse the PDF and convert it to Markdown
os.system(f"""marker_single "{filename}" "../dataTest" --batch_multiplier 1 --ocr_all_pages""")

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 2/2 [00:35<00:00, 17.86s/it]


Saved markdown to the ../dataTest/4355_8432 folder


0

# Header chunking

In [45]:
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
], strip_headers=False)

chunks = markdown_splitter.split_text(doc)
chunks

[Document(page_content='![0_image_0.png](0_image_0.png)  \n![0_image_1.png](0_image_1.png)  \nSCE107727024 Válido até 15/07/2025 Edifício de Habitação IDENTIFICAÇÃO POSTAL\nMorada ESTRADA DAS PEDRAS LAVRADAS, 28, Localidade SOBRAL DE SÃO MIGUEL\nFreguesia SOBRAL DE S. MIGUEL\nConcelho COVILHÃ\nGPS 40.212883, -7.741674 IDENTIFICAÇÃO PREDIAL/FISCAL\nConservatória do Registo Predial de COVILHÃ\nNº de Inscrição na Conservatória 1714 Artigo Matricial nº 637 INFORMAÇÃO ADICIONAL Área útil de Pavimento 100,44 m²\nFração Autónoma Este certificado apresenta a classificação energética deste edificio ou fração. Esta classificação é calculada comparando o desempenho energético deste edificio nas condições atuais, com o desempenho que este obteria nas condições mínimas (com base em valores de referência) a que estão obrigados os edificios novos. Obtenha mais informação sobre a certificação energética no site da ADENE em www.adene.pt INDICADORES DE DESEMPENHO\nDeterminam a classe energética do edifí

In [49]:
len(chunks)

17

# Semantic Chunking

In [37]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY")), breakpoint_threshold_type='percentile', ) # chose which embeddings and breakpoint type and threshold to use
chunks = text_splitter.create_documents([doc])
chunks

[Document(page_content='# Cartório Notarial De Palmela\n\nTelef.:212 350 031 / 212 330 288 - Fax 212 332 542 Av. Rainha D. Leonor, 4 Loja E - 2950 - 204 PALMELA\nNOTÁRIO\nLicenciado Jerónimo Monteiro Lourenço O Signatário, Ajudante do Cartório Notarial de Palmela\n-  Que a fotocópia apensa a esta Certidão está conforme o original que restituí o qual tem / não tem aposto o respectivo selo branco. - Que foi extraída neste Cartório da escritura exarada de folhas per\n-- a folhas ort do livro de notas para escrituras diversas número Que foi extraída neste Cartório do Testamento exarado de folhas _\n_ a folhas _\ndo livro de Testamentos número_\nQue fiz extrair do Bilhete de Identidade número emitido em de de pelos\n- Que fiz extraír do Passaporte número -\nde_\n_ por de de de maine\n..'),
 Document(page_content='- Que me foi presente para conferir. ----------------------\n- Que fiz extraír do documento. -------------------------\n- Que ocupa - Ou 3 l - folhas que têm aposto o respectivo se

# Header + Semantic Chunking?

In [39]:
chunks[0].id

# Basic chunking with overlap

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter

text_splitter = MarkdownTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=40,
    chunk_overlap=0
    # length_function=len,
    # is_separator_regex=False,
    # separator="ch"
)
chunks = text_splitter.create_documents([doc])
chunks

[Document(page_content='![0_image_0.png](0_image_0.png)'),
 Document(page_content='![0_image_1.png](0_image_1.png)'),
 Document(page_content='SCE107727024 Válido até 15/07/2025'),
 Document(page_content='Edifício de Habitação IDENTIFICAÇÃO'),
 Document(page_content='POSTAL'),
 Document(page_content='Morada ESTRADA DAS PEDRAS LAVRADAS, 28,'),
 Document(page_content='Localidade SOBRAL DE SÃO MIGUEL'),
 Document(page_content='Freguesia SOBRAL DE S. MIGUEL'),
 Document(page_content='Concelho COVILHÃ'),
 Document(page_content='GPS 40.212883, -7.741674 IDENTIFICAÇÃO'),
 Document(page_content='PREDIAL/FISCAL'),
 Document(page_content='Conservatória do Registo Predial de'),
 Document(page_content='COVILHÃ'),
 Document(page_content='Nº de Inscrição na Conservatória 1714'),
 Document(page_content='Artigo Matricial nº 637 INFORMAÇÃO'),
 Document(page_content='ADICIONAL Área útil de Pavimento 100,44'),
 Document(page_content='m²'),
 Document(page_content='Fração Autónoma Este certificado'),
 Docum

In [28]:
len(chunks)

1

In [18]:
print(chunks[0].page_content)

da - Sede: Rua Abel Salazar, 7C e 7D, loja 8, piso 0  2905-290 Almada  Tel: 215863163   tipyfamilymo@century21.pt     www.century21.pt/tipy/familymc=Licer

![0_image_0.png](0_image_0.png)

# Century 21

![0_Image_1.Png](0_Image_1.Png)

Ipv Family MC

# Contrato De Colaboração E Partilha De Comissão


In [17]:
print(chunks[1].page_content)

Entre PRIMEIRA CONTRAENTE:


In [16]:
print(chunks[2].page_content)

FGM&C Lda., com sede na Rua Abel Salazar, 7C e 7D loja 8 piso 0, 2805-290 Almada, com o capital social de €5.000, com o NIPC: 516095188, com o código de acesso à certidão comercial n	º 3140-5783-5469, detentora da Licença AMI nº 18194 emitida pelo Instituto dos Mercados Públicos, do Imobiliário e Construção (IMPIC),neste ato representada pela Procuradora Carla Martins, conforme procuração autenticada com o número de registo na Ordem dos Advogados n.º 21535L/3525, adiante designada como Mediadora.: E


In [19]:
print(chunks[3].page_content)

e segunda contraente: 
Obvio e Positivo Loc. Mediação Imobiliária, Lda., com sede na Rua Prof Frencisco Gentil nº 20 - Telheras - Inthuse com capital social
€ 10000 com o NIPC: 514 707 356 detentora da Licença AMI nº 1 6962 emitida pelo Instituto dos Mercados Públicos, do Imobiliário e Construção (IMPIC), neste ato representada pela Por Poulo Costa. I von un dorayante designada como Segunda Outorgante.


In [22]:
print(chunks[4].page_content)

As Contraentes manifestam que é vontade das mesmas subscrever o presente contrato de colaboração que se regerá pelas seguintes cláusulas: -

## Primeira

As partes trocam habitualmente informação confidencial de produtos imobiliários, apresentando os mesmos aos seus clientes, com o objetivo de levar a bom termo operações de caráter imobiliário. -

## Segunda


In [21]:
print(doc)

da - Sede: Rua Abel Salazar, 7C e 7D, loja 8, piso 0  2905-290 Almada  Tel: 215863163   tipyfamilymo@century21.pt     www.century21.pt/tipy/familymc=Licer

![0_image_0.png](0_image_0.png)

# Century 21

![0_Image_1.Png](0_Image_1.Png)

Ipv Family MC

# Contrato De Colaboração E Partilha De Comissão

Entre PRIMEIRA CONTRAENTE:
FGM&C Lda., com sede na Rua Abel Salazar, 7C e 7D loja 8 piso 0, 2805-290 Almada, com o capital social de €5.000, com o NIPC: 516095188, com o código de acesso à certidão comercial n	º 3140-5783-5469, detentora da Licença AMI nº 18194 emitida pelo Instituto dos Mercados Públicos, do Imobiliário e Construção (IMPIC),neste ato representada pela Procuradora Carla Martins, conforme procuração autenticada com o número de registo na Ordem dos Advogados n.º 21535L/3525, adiante designada como Mediadora.: E
e segunda contraente: 
Obvio e Positivo Loc. Mediação Imobiliária, Lda., com sede na Rua Prof Frencisco Gentil nº 20 - Telheras - Inthuse com capital social
€ 10000 co

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.schema.document import Document

In [36]:
load_document("../data/Acordo Partilha Assinado.pdf")
load_document("../data/cpu atualizada.pdf")
load_document("../data/Escritura.pdf")

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 1/1 [00:03<00:00,  3.74s/it]


Saved markdown to the /var/folders/b4/t373qrvd4m76swgs_nb9vf9r0000gn/T/tmp3qzey8im/Acordo Partilha Assinado folder
Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]


Saved markdown to the /var/folders/b4/t373qrvd4m76swgs_nb9vf9r0000gn/T/tmpx58_l1ty/cpu atualizada folder
Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Finding reading order: 100%|██████████| 2/2 [00:21<00:00, 10.68s/it]


Saved markdown to the /var/folders/b4/t373qrvd4m76swgs_nb9vf9r0000gn/T/tmp8svcrv4a/Escritura folder
